In [3]:
# ============================================================
#  Balanceador de (), [], {} para Jupyter (una sola celda)
#  - Lee lab2.txt línea por línea
#  - Muestra pasos de la pila
#  - Instala 'rich' con %pip si no está presente
# ============================================================

# ---------- Auto‑instalador opcional ----------
def asegurar_paquetes(paquetes):
    import importlib, sys, subprocess
    faltantes = [p for p in paquetes if importlib.util.find_spec(p) is None]
    if not faltantes:
        return
    try:                              # ¿Notebook?
        get_ipython().run_line_magic("pip", "install " + " ".join(faltantes))
    except Exception:                 # Script normal
        subprocess.check_call([sys.executable, "-m", "pip", "install", *faltantes])

asegurar_paquetes(["rich"])

# ---------- Imports ----------
try:
    from rich import print as rprint
except Exception:                     # Fallback si rich no se pudo instalar
    def rprint(*a, **k): print(*a, **k)

from pathlib import Path
from dataclasses import dataclass
from typing import List, Optional

# ---------- Constantes ----------
PARES = {')': '(', ']': '[', '}': '{'}
ABRE  = set(PARES.values())
CIERRA = set(PARES.keys())

# ---------- Dataclasses ----------
@dataclass
class PasoPila:
    paso: int
    pos_char: int
    char: str
    accion: str          # 'push', 'pop', 'pop (vacía)'
    pila_resultado: str

@dataclass
class ResultadoLinea:
    linea_num: int
    expresion: str
    balanceada: bool
    mensaje: str
    pasos: List[PasoPila]

# ---------- Lógica principal ----------
def inversa(simbolo_apertura: str) -> str:
    for k, v in PARES.items():
        if v == simbolo_apertura:
            return k
    return '?'

def balancear_expresion(expresion: str, linea_num: int,
                        mostrar_pasos: bool = True) -> ResultadoLinea:
    pila: List[str] = []
    pasos: List[PasoPila] = []
    error_msg: Optional[str] = None
    paso_id = 1

    for i, ch in enumerate(expresion, start=1):
        if ch in ABRE:
            pila.append(ch)
            if mostrar_pasos:
                pasos.append(PasoPila(paso_id, i, ch, 'push', ''.join(pila))); paso_id += 1
        elif ch in CIERRA:
            if pila:
                tope = pila.pop()
                if mostrar_pasos:
                    pasos.append(PasoPila(paso_id, i, ch, 'pop', ''.join(pila))); paso_id += 1
                if tope != PARES[ch]:
                    error_msg = (f"Error: se esperaba cerrar '{inversa(tope)}' "
                                 f"pero apareció '{ch}' en la posición {i}.")
                    break
            else:
                if mostrar_pasos:
                    pasos.append(PasoPila(paso_id, i, ch, 'pop (vacía)', ''.join(pila))); paso_id += 1
                error_msg = f"Error: se encontró '{ch}' en la posición {i} pero la pila estaba vacía."
                break

    if error_msg is None and pila:
        faltantes = ''.join(inversa(s) for s in reversed(pila))
        error_msg = f"Error: falta(n) cerrar {faltantes}. La pila no quedó vacía."

    balanceada = error_msg is None
    mensaje = "Expresión balanceada." if balanceada else error_msg
    return ResultadoLinea(linea_num, expresion, balanceada, mensaje, pasos)

def imprimir_resultado(res: ResultadoLinea, mostrar_pasos: bool = True):
    rprint(f"\n[bold cyan]Línea {res.linea_num}:[/bold cyan] {res.expresion}")
    rprint("Resultado:", "[green]✔ Balanceada[/green]" if res.balanceada else "[red]✘ NO balanceada[/red]")
    rprint("Detalle:", res.mensaje)
    if mostrar_pasos:
        rprint("\n  [bold]Pasos de la pila:[/bold]")
        rprint("  {:>4} | {:>6} | {:>6} | {:>12} | {}".format("Paso", "Pos", "Char", "Acción", "Pila después"))
        rprint("  " + "-" * 64)
        for p in res.pasos:
            rprint(f"  {p.paso:>4} | {p.pos_char:>6} | {p.char:>6} | {p.accion:>12} | {p.pila_resultado}")

def leer_y_procesar(ruta_archivo: Path, mostrar_pasos: bool = True):
    resultados = []
    with ruta_archivo.open(encoding='utf-8') as f:
        for n_linea, linea in enumerate(f, start=1):
            expr = linea.rstrip('\n')
            if expr.strip():
                resultados.append(balancear_expresion(expr, n_linea, mostrar_pasos))
    return resultados

def resumen_final(resultados):
    rprint("\n" + "=" * 72)
    rprint("[bold magenta]RESUMEN[/bold magenta]")
    rprint("=" * 72)
    for r in resultados:
        estado = "[green]OK[/green]" if r.balanceada else "[red]ERROR[/red]"
        rprint(f"Línea {r.linea_num:>2}: {estado} - {r.mensaje}")
    return 0 if all(r.balanceada for r in resultados) else 1

# ---------- Función que usaremos en el notebook ----------
def run_balanceador(path_archivo: str, mostrar_pasos: bool = True):
    ruta = Path(path_archivo)
    if not ruta.exists():
        rprint(f"[red]El archivo '{ruta}' no existe.[/red]")
        return
    resultados = leer_y_procesar(ruta, mostrar_pasos)
    for res in resultados:
        imprimir_resultado(res, mostrar_pasos)
    code = resumen_final(resultados)
    rprint(f"\nCódigo de salida simulado: {code}")

# ---------- EJECUCIÓN AUTOMÁTICA SOBRE lab2.txt ----------
run_balanceador("lab2.txt", mostrar_pasos=True)  # cambia a False si no quieres ver la traza


Línea 1: 𝑎(𝑎|𝑏) ∗ 𝑏 + 𝑎?

Resultado: ✔ Balanceada

Detalle: Expresión balanceada.

Pasos de la pila:

Paso |    Pos |   Char |       Acción | Pila después

...

Línea  2: ERROR - Error: se encontró ']' en la posición 17 pero la pila estaba vacía.

Línea  3: ERROR - Error: se encontró ')' en la posición 43 pero la pila estaba vacía.

Línea  4: OK - Expresión balanceada.

Línea  5: ERROR - Error: se encontró ')' en la posición 49 pero la pila estaba vacía.

Código de salida simulado: 1

In [13]:
# ============================================================
#  Shunting‑Yard (versión robusta) con manejo de errores
# ============================================================
from pathlib import Path
from dataclasses import dataclass
from typing import List

# ---------- Precedencias ----------
PREC = {'|': 2, '·': 3, '?': 4, '*': 4, '+': 4, '^': 5, '(': 1}

# ---------- Dataclasses ----------
@dataclass
class PasoSY:
    paso  : int
    token : str
    accion: str          # push / pop / output / error
    pila  : str
    salida: str

@dataclass
class ResultadoSY:
    linea     : int
    original  : str
    postfix   : str
    error_msg : str | None
    pasos     : List[PasoSY]

# ---------- Utilidades ----------
def es_operador(tok: str) -> bool:
    return tok in PREC and tok != '('

def tokenizar(regex: str) -> List[str]:
    tokens = []
    i = 0
    while i < len(regex):
        if regex[i] == '\\' and i + 1 < len(regex):
            tokens.append(regex[i:i+2]); i += 2
        else:
            tokens.append(regex[i]);    i += 1
    return tokens

def necesita_concat(c1: str, c2: str) -> bool:
    if c1 == '(' or c2 == ')' : return False
    if c2 == '|'              : return False
    if es_operador(c1) and PREC[c1] != 4: return False
    return True

def format_regex(regex: str) -> List[str]:
    toks = tokenizar(regex)
    if not toks: return []
    res = []
    for i, t1 in enumerate(toks[:-1]):
        t2 = toks[i+1]
        res.append(t1)
        if necesita_concat(t1[-1], t2[0]):
            res.append('·')
    res.append(toks[-1])
    return res

# ---------- Algoritmo principal ----------
def infix_to_postfix(regex: str, idx_linea: int) -> ResultadoSY:
    tokens = format_regex(regex)
    salida: List[str] = []
    pila  : List[str] = []
    pasos : List[PasoSY] = []
    step  = 1
    error = None

    def push(tok):
        nonlocal step
        pila.append(tok)
        pasos.append(PasoSY(step, tok, 'push', ''.join(pila), ''.join(salida))); step += 1

    def pop_to_output():
        nonlocal step
        tok = pila.pop()
        salida.append(tok)
        pasos.append(PasoSY(step, tok, 'pop→out', ''.join(pila), ''.join(salida))); step += 1

    for tok in tokens:
        if error: break

        if tok == '(':
            push(tok)

        elif tok == ')':
            if not pila:
                error = "Paréntesis de cierre sin apertura."
                pasos.append(PasoSY(step, tok, 'error', ''.join(pila), ''.join(salida))); step += 1
                break
            while pila and pila[-1] != '(':
                pop_to_output()
                if not pila:  # nunca apareció '('
                    error = "Paréntesis de cierre sin apertura."
                    pasos.append(PasoSY(step, tok, 'error', ''.join(pila), ''.join(salida))); step += 1
                    break
            if error: break
            pila.pop()  # descartar '('
            pasos.append(PasoSY(step, ')', 'pop (', ''.join(pila), ''.join(salida))); step += 1

        elif es_operador(tok):
            while pila and es_operador(pila[-1]) and PREC[pila[-1]] >= PREC[tok]:
                pop_to_output()
            push(tok)

        else:  # literal
            salida.append(tok)
            pasos.append(PasoSY(step, tok, 'output', ''.join(pila), ''.join(salida))); step += 1

    if not error:
        while pila and pila[-1] != '(':
            pop_to_output()
        if pila and pila[-1] == '(':
            error = "Paréntesis de apertura sin cierre."
            pasos.append(PasoSY(step, '', 'error', ''.join(pila), ''.join(salida))); step += 1

    postfix = ''.join(salida)
    return ResultadoSY(idx_linea, regex, postfix, error, pasos)

# ---------- Función de usuario ----------
def run_shunting(path_file: str, mostrar_pasos: bool = True):
    ruta = Path(path_file)
    if not ruta.exists():
        print(f"❌ El archivo {ruta} no existe."); return

    for idx, linea in enumerate(ruta.read_text(encoding='utf-8').splitlines(), 1):
        expr = linea.strip()
        if not expr: continue
        res = infix_to_postfix(expr, idx)
        print("\n" + "═"*60)
        print(f"Línea {idx}:  {expr}")
        if res.error_msg:
            print(f"❌  Error: {res.error_msg}")
        else:
            print(f"Postfix :  {res.postfix}")
        if mostrar_pasos:
            print("Pasos:")
            print("Paso | Token | Acción     | Pila        | Salida")
            print("-----+-------+------------+-------------+----------------")
            for p in res.pasos:
                print(f"{p.paso:>4} | {p.token:<5} | {p.accion:<10} | {p.pila:<11} | {p.salida}")
run_shunting("lab2.txt", mostrar_pasos=True)



════════════════════════════════════════════════════════════
Línea 1:  𝑎(𝑎|𝑏) ∗ 𝑏 + 𝑎?
Postfix :  𝑎𝑎𝑏|· ·∗· ·𝑏· ·+· ·𝑎·?·
Pasos:
Paso | Token | Acción     | Pila        | Salida
-----+-------+------------+-------------+----------------
   1 | 𝑎     | output     |             | 𝑎
   2 | ·     | push       | ·           | 𝑎
   3 | (     | push       | ·(          | 𝑎
   4 | 𝑎     | output     | ·(          | 𝑎𝑎
   5 | |     | push       | ·(|         | 𝑎𝑎
   6 | 𝑏     | output     | ·(|         | 𝑎𝑎𝑏
   7 | |     | pop→out    | ·(          | 𝑎𝑎𝑏|
   8 | )     | pop (      | ·           | 𝑎𝑎𝑏|
   9 | ·     | pop→out    |             | 𝑎𝑎𝑏|·
  10 | ·     | push       | ·           | 𝑎𝑎𝑏|·
  11 |       | output     | ·           | 𝑎𝑎𝑏|· 
  12 | ·     | pop→out    |             | 𝑎𝑎𝑏|· ·
  13 | ·     | push       | ·           | 𝑎𝑎𝑏|· ·
  14 | ∗     | output     | ·           | 𝑎𝑎𝑏|· ·∗
  15 | ·     | pop→out    |             | 𝑎𝑎𝑏|· ·∗·
  16 | ·     | push       | ·           | 𝑎𝑎𝑏|· ·∗·
 